In [2]:
import os
import random
import numpy as np
import pandas as pd
from pathlib import Path
import json
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.cuda.amp import autocast, GradScaler

from PIL import Image
import torchvision.transforms as T

import timm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, balanced_accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Set seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Device: cpu
PyTorch version: 2.1.0+cpu
CUDA available: False


In [3]:
# Cell 2: Configure Paths & Validate Files

import os
import pandas as pd

# Configure paths
PATH_TRAIN_CSV = r"D:\ISIC2018\GroundTruth\Training_GrounTruth\ISIC2018_Task3_Training_GroundTruth.csv"
PATH_VAL_CSV = r"D:\ISIC2018\GroundTruth\Validation_GroundTruth\ISIC2018_Task3_Validation_GroundTruth.csv"
PATH_TEST_CSV = r"D:\ISIC2018\GroundTruth\Test_GroundTruth\ISIC2018_Task3_Test_GroundTruth.csv"

# Image directories
DIR_TRAIN_IMG = r"D:\ISIC2018\Training_Input"
DIR_VAL_IMG = r"D:\ISIC2018\Validation_Input"
DIR_TEST_IMG = r"D:\ISIC2018\Test_Input"

PATH_LESION_GROUPINGS = r"D:\ISIC2018\Training_LesionGroupings.csv"

# Create output directories
os.makedirs("outputs/models", exist_ok=True)
os.makedirs("outputs/submissions", exist_ok=True)
os.makedirs("outputs/figures", exist_ok=True)


# Read and preview CSVs
print("\n=== TRAIN CSV (first 5 rows) ===")
df_train_raw = pd.read_csv(PATH_TRAIN_CSV)
print(df_train_raw.head())
print(f"Train CSV has {len(df_train_raw)} entries")

print("\n=== VAL CSV (first 5 rows) ===")
df_val_raw = pd.read_csv(PATH_VAL_CSV)
print(df_val_raw.head())
print(f"Val CSV has {len(df_val_raw)} entries")

print("\n=== TEST CSV (first 5 rows) ===")
df_test_raw = pd.read_csv(PATH_TEST_CSV)
print(df_test_raw.head())
print(f"Test CSV has {len(df_test_raw)} entries")


=== TRAIN CSV (first 5 rows) ===
          image  MEL   NV  BCC  AKIEC  BKL   DF  VASC
0  ISIC_0024306  0.0  1.0  0.0    0.0  0.0  0.0   0.0
1  ISIC_0024307  0.0  1.0  0.0    0.0  0.0  0.0   0.0
2  ISIC_0024308  0.0  1.0  0.0    0.0  0.0  0.0   0.0
3  ISIC_0024309  0.0  1.0  0.0    0.0  0.0  0.0   0.0
4  ISIC_0024310  1.0  0.0  0.0    0.0  0.0  0.0   0.0
Train CSV has 10015 entries

=== VAL CSV (first 5 rows) ===
          image  MEL   NV  BCC  AKIEC  BKL   DF  VASC
0  ISIC_0034321  0.0  1.0  0.0    0.0  0.0  0.0   0.0
1  ISIC_0034322  0.0  1.0  0.0    0.0  0.0  0.0   0.0
2  ISIC_0034323  0.0  0.0  1.0    0.0  0.0  0.0   0.0
3  ISIC_0034324  0.0  1.0  0.0    0.0  0.0  0.0   0.0
4  ISIC_0034325  0.0  1.0  0.0    0.0  0.0  0.0   0.0
Val CSV has 193 entries

=== TEST CSV (first 5 rows) ===
          image  MEL   NV  BCC  AKIEC  BKL   DF  VASC
0  ISIC_0034524  0.0  1.0  0.0    0.0  0.0  0.0   0.0
1  ISIC_0034525  0.0  1.0  0.0    0.0  0.0  0.0   0.0
2  ISIC_0034526  0.0  0.0  0.0    0.0  

In [4]:
# Cell 3: Parse Ground Truth & Create Label Mapping
def parse_ground_truth(df_raw, img_dir, split_name="train"):

    df = df_raw.copy()
    
    # Detect image_id column (first column or contains 'image')
    img_col = df.columns[0]
    
    # Detect label columns (one-hot encoded or single label column)
    # ISIC2018 Task 3 uses one-hot encoding for 7 disease categories
    potential_label_cols = [col for col in df.columns if col != img_col]
    
    # Check if one-hot encoded (multiple binary columns)
    if len(potential_label_cols) > 1 and df[potential_label_cols].isin([0, 1]).all().all():
        # One-hot encoded: find the column with 1
        df['label'] = df[potential_label_cols].idxmax(axis=1)
    else:
        # Single label column
        df['label'] = df[potential_label_cols[0]]
    
    # Create image_id and image_path
    df['image_id'] = df[img_col].astype(str)
    df['image_path'] = df['image_id'].apply(lambda x: os.path.join(img_dir, f"{x}.jpg"))
    
    # Select relevant columns
    df_clean = df[['image_id', 'label', 'image_path']].copy()
    
    print(f"\n=== {split_name.upper()} DATASET ===")
    print(f"Total samples: {len(df_clean)}")
    print(f"\nLabel distribution:")
    label_counts = df_clean['label'].value_counts().sort_index()
    for label, count in label_counts.items():
        pct = 100 * count / len(df_clean)
        print(f"  {label}: {count} ({pct:.2f}%)")
    
    return df_clean

# Parse all splits
df_train = parse_ground_truth(df_train_raw, DIR_TRAIN_IMG, "train")
df_val = parse_ground_truth(df_val_raw, DIR_VAL_IMG, "val")
df_test = parse_ground_truth(df_test_raw, DIR_TEST_IMG, "test")

# Create label mapping
all_labels = sorted(df_train['label'].unique())
label2idx = {label: idx for idx, label in enumerate(all_labels)}
idx2label = {idx: label for label, idx in label2idx.items()}
num_classes = len(all_labels)

print(f"\n=== LABEL MAPPING ===")
print(f"Number of classes: {num_classes}")
for label, idx in label2idx.items():
    print(f"  {idx}: {label}")

# Apply label mapping
df_train['label_idx'] = df_train['label'].map(label2idx)
df_val['label_idx'] = df_val['label'].map(label2idx)
df_test['label_idx'] = df_test['label'].map(label2idx)

# Check for class imbalance
train_label_counts = df_train['label_idx'].value_counts().sort_index()
max_count = train_label_counts.max()
min_count = train_label_counts.min()
imbalance_ratio = max_count / min_count
print(f"\nClass imbalance ratio: {imbalance_ratio:.2f}")
if imbalance_ratio > 3:
    print("⚠ High class imbalance detected. Will use WeightedRandomSampler.")
    USE_WEIGHTED_SAMPLER = True
else:
    USE_WEIGHTED_SAMPLER = False


=== TRAIN DATASET ===
Total samples: 10015

Label distribution:
  AKIEC: 327 (3.27%)
  BCC: 514 (5.13%)
  BKL: 1099 (10.97%)
  DF: 115 (1.15%)
  MEL: 1113 (11.11%)
  NV: 6705 (66.95%)
  VASC: 142 (1.42%)

=== VAL DATASET ===
Total samples: 193

Label distribution:
  AKIEC: 8 (4.15%)
  BCC: 15 (7.77%)
  BKL: 22 (11.40%)
  DF: 1 (0.52%)
  MEL: 21 (10.88%)
  NV: 123 (63.73%)
  VASC: 3 (1.55%)

=== TEST DATASET ===
Total samples: 1512

Label distribution:
  AKIEC: 43 (2.84%)
  BCC: 93 (6.15%)
  BKL: 217 (14.35%)
  DF: 44 (2.91%)
  MEL: 171 (11.31%)
  NV: 909 (60.12%)
  VASC: 35 (2.31%)

=== LABEL MAPPING ===
Number of classes: 7
  0: AKIEC
  1: BCC
  2: BKL
  3: DF
  4: MEL
  5: NV
  6: VASC

Class imbalance ratio: 58.30
⚠ High class imbalance detected. Will use WeightedRandomSampler.


In [5]:
# Cell 4: Dataset Class
class ISICDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        label = row['label_idx']
        image_id = row['image_id']
        
        # Load image
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image not found: {image_path}")
        
        image = Image.open(image_path).convert('RGB')
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label, dtype=torch.long), image_id

In [6]:
# Cell 5: Transforms
# EfficientNet-B3 default input size is 300
IMG_SIZE = 300

# ImageNet normalization
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# Train transforms with augmentation
train_transform = T.Compose([
    T.Resize(320),
    T.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomVerticalFlip(p=0.5),
    T.RandomRotation(20),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),  # Giảm hue từ 0.1 xuống 0.05
    T.ToTensor(),
    T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# Val/Test transforms without augmentation
val_transform = T.Compose([
    T.Resize(320),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

print("Train transforms:", train_transform)
print("\nVal/Test transforms:", val_transform)

Train transforms: Compose(
    Resize(size=320, interpolation=bilinear, max_size=None, antialias=warn)
    RandomResizedCrop(size=(300, 300), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
    RandomHorizontalFlip(p=0.5)
    RandomVerticalFlip(p=0.5)
    RandomRotation(degrees=[-20.0, 20.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.05, 0.05))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

Val/Test transforms: Compose(
    Resize(size=320, interpolation=bilinear, max_size=None, antialias=warn)
    CenterCrop(size=(300, 300))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [7]:
# Cell 6: DataLoaders
# Batch size configuration
BATCH_SIZE = 16 if torch.cuda.is_available() else 8
NUM_WORKERS = 0  # Set to 0 for Windows to avoid issues

# Create datasets
train_dataset = ISICDataset(df_train, transform=train_transform)
val_dataset = ISICDataset(df_val, transform=val_transform)
test_dataset = ISICDataset(df_test, transform=val_transform)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Create WeightedRandomSampler if imbalanced
if USE_WEIGHTED_SAMPLER:
    class_counts = df_train['label_idx'].value_counts().sort_index().values
    class_weights = 1.0 / class_counts
    sample_weights = [class_weights[label] for label in df_train['label_idx']]
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights))
    shuffle = False
    print("Using WeightedRandomSampler for balanced sampling")
else:
    sampler = None
    shuffle = True

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    sampler=sampler,
    shuffle=shuffle,
    num_workers=NUM_WORKERS,
    pin_memory=torch.cuda.is_available()
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=torch.cuda.is_available()
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=torch.cuda.is_available()
)

print(f"\nBatch size: {BATCH_SIZE}")
print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

Train dataset size: 10015
Val dataset size: 193
Test dataset size: 1512
Using WeightedRandomSampler for balanced sampling

Batch size: 8
Train batches: 1252
Val batches: 25
Test batches: 189


In [8]:
# Cell 7: Model Architecture
def build_model(num_classes, pretrained=True):
    """Build EfficientNet-B3 model with custom classifier."""
    model = timm.create_model('efficientnet_b3', pretrained=pretrained, num_classes=num_classes)
    return model

# Create model
model = build_model(num_classes, pretrained=True)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model: EfficientNet-B3")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Number of classes: {num_classes}")

Model: EfficientNet-B3
Total parameters: 10,706,991
Trainable parameters: 10,706,991
Number of classes: 7


In [9]:
# Cell 8: Loss, Optimizer, Scheduler
# Configuration
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 1e-4
USE_LABEL_SMOOTHING = False  # Set to True if desired
LABEL_SMOOTHING = 0.1

# Loss function
if USE_LABEL_SMOOTHING:
    criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)
    print(f"Using CrossEntropyLoss with label smoothing: {LABEL_SMOOTHING}")
else:
    criterion = nn.CrossEntropyLoss()
    print("Using CrossEntropyLoss")

# Optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)
print(f"Optimizer: AdamW (lr={LEARNING_RATE}, weight_decay={WEIGHT_DECAY})")

# Scheduler
USE_COSINE_SCHEDULER = True  # Set to False to use ReduceLROnPlateau
NUM_EPOCHS = 15

if USE_COSINE_SCHEDULER:
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=NUM_EPOCHS, eta_min=1e-6
    )
    print(f"Scheduler: CosineAnnealingLR (T_max={NUM_EPOCHS})")
else:
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2, verbose=True
    )
    print("Scheduler: ReduceLROnPlateau (mode=max, factor=0.5, patience=2)")

Using CrossEntropyLoss
Optimizer: AdamW (lr=0.0003, weight_decay=0.0001)
Scheduler: CosineAnnealingLR (T_max=15)


In [ ]:
# Cell 9: Train & Evaluation Functions
def train_one_epoch(model, loader, criterion, optimizer, device, scaler=None):
    """Train for one epoch."""
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    for images, labels, _ in loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        # Mixed precision training
        if scaler is not None:
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    
    return epoch_loss, epoch_acc


def evaluate(model, loader, criterion, device):
    """Evaluate model on validation/test set."""
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    all_probs = []
    all_image_ids = []
    
    with torch.no_grad():
        for images, labels, image_ids in loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            probs = F.softmax(outputs, dim=1).cpu().numpy()
            preds = outputs.argmax(dim=1).cpu().numpy()
            
            all_probs.extend(probs)
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
            all_image_ids.extend(image_ids)
    
    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    epoch_f1 = f1_score(all_labels, all_preds, average='macro')
    epoch_bal_acc = balanced_accuracy_score(all_labels, all_preds)
    
    return epoch_loss, epoch_acc, epoch_f1, epoch_bal_acc, all_preds, all_labels, all_probs, all_image_ids


print("Train and evaluation functions defined.")

Train and evaluation functions defined.


: 

In [ ]:
# Cell 10: Training Loop with Checkpoint Resume
from tqdm.notebook import tqdm
import sys

# Training configuration
EARLY_STOP_PATIENCE = 3
MONITOR_METRIC = 'val_f1'
MODEL_PATH = 'outputs/models/efficientnet_b3_isic2018.pt'

# Initialize mixed precision scaler
scaler = GradScaler() if torch.cuda.is_available() else None

# Training history
history = {
    'epoch': [], 'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': [], 'val_f1': [], 'val_bal_acc': [], 'lr': []
}

# Best model tracking
best_val_f1 = 0.0
best_epoch = 0
patience_counter = 0
start_epoch = 0

# ========== LOAD EXISTING CHECKPOINT IF EXISTS ==========
if os.path.exists(MODEL_PATH):
    print(f"📂 Found existing checkpoint: {MODEL_PATH}")
    checkpoint = torch.load(MODEL_PATH, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint.get('epoch', 0)
    best_val_f1 = checkpoint.get('best_val_f1', 0.0)
    best_epoch = checkpoint.get('best_epoch', 0)
    if 'history' in checkpoint:
        history = checkpoint['history']
    print(f"✓ Resumed from epoch {start_epoch}, best F1: {best_val_f1:.4f}")
else:
    print("🆕 Starting fresh training (no checkpoint found)")

print(f"\nStarting training for {NUM_EPOCHS} epochs (from epoch {start_epoch + 1})...")
print(f"Early stopping: patience={EARLY_STOP_PATIENCE}, monitor={MONITOR_METRIC}\n")

for epoch in range(start_epoch, NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    print("-" * 50)
    sys.stdout.flush()
    
    # Train with progress bar
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    train_pbar = tqdm(train_loader, desc=f"Training", leave=True)
    for images, labels, _ in train_pbar:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        if scaler is not None:
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())
        train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    train_loss = running_loss / len(train_loader.dataset)
    train_acc = accuracy_score(all_labels, all_preds)
    
    # Validate
    val_loss, val_acc, val_f1, val_bal_acc, _, _, _, _ = evaluate(model, val_loader, criterion, device)
    
    # Update learning rate
    current_lr = optimizer.param_groups[0]['lr']
    if USE_COSINE_SCHEDULER:
        scheduler.step()
    else:
        scheduler.step(val_f1)
    
    # Record history
    history['epoch'].append(epoch + 1)
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    history['val_bal_acc'].append(val_bal_acc)
    history['lr'].append(current_lr)
    
    # Print epoch summary
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")
    print(f"Learning Rate: {current_lr:.6f}")
    sys.stdout.flush()
    
    # Check for best model and ALWAYS save checkpoint
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_epoch = epoch + 1
        patience_counter = 0
        print(f"✓ New best model! (F1: {val_f1:.4f})")
    else:
        patience_counter += 1
        print(f"No improvement. Patience: {patience_counter}/{EARLY_STOP_PATIENCE}")
    
    # ========== SAVE CHECKPOINT (ALWAYS UPDATE) ==========
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_val_f1': best_val_f1,
        'best_epoch': best_epoch,
        'val_f1': val_f1,
        'val_acc': val_acc,
        'history': history,
        'label2idx': label2idx,
        'idx2label': idx2label,
        'num_classes': num_classes,
    }, MODEL_PATH)
    print(f"💾 Checkpoint saved: {MODEL_PATH}")
    sys.stdout.flush()
    
    # Early stopping
    if patience_counter >= EARLY_STOP_PATIENCE:
        print(f"\n Early stopping triggered at epoch {epoch + 1}")
        break

print(f"\n{'='*50}")
print(f"✅ Training completed!")
print(f"📊 Best epoch: {best_epoch} with Val F1: {best_val_f1:.4f}")
print(f"📁 Model saved: {MODEL_PATH}")

🆕 Starting fresh training (no checkpoint found)

Starting training for 15 epochs (from epoch 1)...
Early stopping: patience=3, monitor=val_f1


Epoch 1/15
--------------------------------------------------


Training:   0%|          | 0/1252 [00:00<?, ?it/s]

In [ ]:
# Cell 11: Evaluate on Validation
# Load best model
print("Loading best model...")
checkpoint = torch.load('outputs/models/best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded model from epoch {checkpoint['epoch']} with best metric: {checkpoint['best_metric']:.4f}\n")

# Evaluate on validation set
print("Evaluating on validation set...")
val_loss, val_acc, val_f1, val_bal_acc, val_preds, val_labels, val_probs, val_image_ids = evaluate(
    model, val_loader, criterion, device
)

print(f"Validation Results:")
print(f"  Loss: {val_loss:.4f}")
print(f"  Accuracy: {val_acc:.4f}")
print(f"  Macro F1: {val_f1:.4f}")
print(f"  Balanced Accuracy: {val_bal_acc:.4f}")

# Confusion Matrix
cm = confusion_matrix(val_labels, val_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=[idx2label[i] for i in range(num_classes)],
            yticklabels=[idx2label[i] for i in range(num_classes)])
plt.title('Validation Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('outputs/figures/val_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Classification Report
print("\n=== Classification Report ===")
print(classification_report(val_labels, val_preds, 
                          target_names=[idx2label[i] for i in range(num_classes)],
                          digits=4))

# Show misclassified examples
print("\n=== Misclassified Examples ===")
val_results_df = pd.DataFrame({
    'image_id': val_image_ids,
    'true_label': val_labels,
    'pred_label': val_preds,
})
val_results_df['true_class'] = val_results_df['true_label'].map(idx2label)
val_results_df['pred_class'] = val_results_df['pred_label'].map(idx2label)
val_results_df['correct'] = val_results_df['true_label'] == val_results_df['pred_label']

misclassified = val_results_df[~val_results_df['correct']].head(16)
print(f"Found {len(val_results_df[~val_results_df['correct']])} misclassified images")

if len(misclassified) > 0:
    fig, axes = plt.subplots(4, 4, figsize=(16, 16))
    axes = axes.flatten()
    
    for idx, (_, row) in enumerate(misclassified.iterrows()):
        if idx >= 16:
            break
        
        img_path = os.path.join(DIR_VAL_IMG, f"{row['image_id']}.jpg")
        if os.path.exists(img_path):
            img = Image.open(img_path).convert('RGB')
            axes[idx].imshow(img)
            axes[idx].set_title(f"True: {row['true_class']}\nPred: {row['pred_class']}", 
                               fontsize=10)
            axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig('outputs/figures/val_misclassified.png', dpi=150, bbox_inches='tight')
    plt.show()

Loading best model...


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/models/best_model.pth'

In [ ]:
# Cell 12: Evaluate on Test
# Evaluate on test set
print("Evaluating on test set...")
test_loss, test_acc, test_f1, test_bal_acc, test_preds, test_labels, test_probs, test_image_ids = evaluate(
    model, test_loader, criterion, device
)

print(f"Test Results:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_acc:.4f}")
print(f"  Macro F1: {test_f1:.4f}")
print(f"  Balanced Accuracy: {test_bal_acc:.4f}")

# Confusion Matrix
cm_test = confusion_matrix(test_labels, test_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Greens',
            xticklabels=[idx2label[i] for i in range(num_classes)],
            yticklabels=[idx2label[i] for i in range(num_classes)])
plt.title('Test Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('outputs/figures/test_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Classification Report
print("\n=== Test Classification Report ===")
print(classification_report(test_labels, test_preds,
                          target_names=[idx2label[i] for i in range(num_classes)],
                          digits=4))

# Create submission file
submission_data = {
    'image_id': test_image_ids,
    'predicted_label': [idx2label[pred] for pred in test_preds],
}

# Add probability columns for each class
for i in range(num_classes):
    submission_data[f'prob_{idx2label[i]}'] = [probs[i] for probs in test_probs]

submission_df = pd.DataFrame(submission_data)
submission_path = 'outputs/submissions/test_predictions.csv'
submission_df.to_csv(submission_path, index=False)
print(f"\nSubmission file saved to: {submission_path}")
print(f"Submission shape: {submission_df.shape}")
print("\nFirst 5 predictions:")
print(submission_df.head())

In [ ]:
# Cell 13: Inference Demo
def predict_image(model, image_path, transform, device, idx2label, top_k=3):
    """Predict single image and return top-k predictions."""
    model.eval()
    
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Predict
    with torch.no_grad():
        outputs = model(image_tensor)
        probs = F.softmax(outputs, dim=1).cpu().numpy()[0]
    
    # Get top-k predictions
    top_k_indices = np.argsort(probs)[::-1][:top_k]
    top_k_probs = probs[top_k_indices]
    top_k_labels = [idx2label[idx] for idx in top_k_indices]
    
    return image, top_k_labels, top_k_probs


# Demo images
demo_images = [
    (r"D:\ISIC2018\Test_Input\ISIC_0034524.jpg", "ISIC_0034524"),
    (r"D:\ISIC2018\Validation_Input\ISIC_0034321.jpg", "ISIC_0034321"),
]

print("=== Inference Demo ===\n")

fig, axes = plt.subplots(1, len(demo_images), figsize=(12, 5))
if len(demo_images) == 1:
    axes = [axes]

for idx, (img_path, img_name) in enumerate(demo_images):
    if os.path.exists(img_path):
        image, top_labels, top_probs = predict_image(
            model, img_path, val_transform, device, idx2label, top_k=3
        )
        
        # Display
        axes[idx].imshow(image)
        title = f"{img_name}\n"
        for i, (label, prob) in enumerate(zip(top_labels, top_probs)):
            title += f"{i+1}. {label}: {prob:.3f}\n"
        axes[idx].set_title(title, fontsize=9)
        axes[idx].axis('off')
        
        # Print to console
        print(f"Image: {img_name}")
        for i, (label, prob) in enumerate(zip(top_labels, top_probs)):
            print(f"  {i+1}. {label}: {prob:.4f} ({prob*100:.2f}%)")
        print()
    else:
        print(f"Image not found: {img_path}\n")

plt.tight_layout()
plt.savefig('outputs/figures/inference_demo.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Inference demo completed!")